# Basic cleaning

In [1]:
import pandas as pd
import numpy as np

from typing import List

from sklearn.impute import SimpleImputer

from util.cleaners import BasicCleaner

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [3]:
# Remove capitalization on column names
train_df.columns = [column_name.lower() for column_name in train_df.columns]
test_df.columns = [column_name.lower() for column_name in test_df.columns]

### Apply basic cleaning and feature extraction

In [4]:
cleaner = BasicCleaner()
train_cleaned = cleaner.preprocessing(train_df)
test_cleaned = cleaner.preprocessing(test_df)

In [5]:
train_cleaned.head(3)

,passengerid,survived,pclass,sex,age,sibsp,parch,ticket,fare,cabin,embarked,first_name,last_name,title
0,1,0,3,1,22.0,1,0,A/5 21171,7.2500,NaN,S,Braund,Owen Harris,mr
1,2,1,1,0,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,John Bradley (Florence Briggs Thayer),mrs
2,3,1,3,0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Heikkinen,Laina,miss


#### To CSV

In [6]:
train_cleaned.to_csv('data/train_cleaned_basic.csv', index=False)
test_cleaned.to_csv('data/test_cleaned_basic.csv', index=False)